In [84]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
import pandas as pd

Importing the Dataset

In [85]:
from sklearn.datasets import make_circles

In [86]:
make_circles

<function sklearn.datasets._samples_generator.make_circles(n_samples=100, *, shuffle=True, noise=None, random_state=None, factor=0.8)>

In [87]:
n_samples = 1000

Defining the input and output variable

In [88]:
X,y = make_circles(n_samples , noise = 0.03 , shuffle = True )

In [89]:
X

array([[-0.65241787,  0.49493262],
       [-0.74031977, -0.12706414],
       [-0.7511088 , -0.31441569],
       ...,
       [-0.07305639,  0.99167301],
       [ 0.23446096, -0.9473844 ],
       [-0.88792217, -0.48912496]])

In [90]:
y

array([1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,

Preparing the DataFrame

In [91]:
circles_df = pd.DataFrame({"X1" : X[:,0] , "X2" : X[:,1] ,"label" : y})

In [92]:
circles_df.head(10)

,X1,X2,label
0,-0.652418,0.494933,1
1,-0.740320,-0.127064,1
2,-0.751109,-0.314416,1
3,0.776068,-0.140823,1
4,0.689250,0.722976,0
5,-0.520514,0.913144,0
6,-0.822543,-0.500040,0
7,0.169276,-0.787709,1
8,0.385830,0.878167,0
9,-0.379821,-0.883063,0


 Identifying the shape of the Dataset

In [93]:
print(f"The shape of input feature is {X.shape} , Shape of target feature is {y.shape}")

The shape of input feature is (1000, 2) , Shape of target feature is (1000,)


# 1 Turning our data to Tensors and creating the splits

In [94]:
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

In [95]:
X [:5] , y[:5]

(tensor([[-0.6524,  0.4949],
         [-0.7403, -0.1271],
         [-0.7511, -0.3144],
         [ 0.7761, -0.1408],
         [ 0.6892,  0.7230]]),
 tensor([1., 1., 1., 1., 0.]))

In [96]:
type(X) , X.dtype , type(y)

(torch.Tensor, torch.float32, torch.Tensor)

Splitting the data

In [97]:
from sklearn.model_selection import train_test_split
x_train , x_test = train_test_split(X , random_state = 42 , test_size = 0.2)
y_train , y_test = train_test_split(y , random_state = 42 , test_size = 0.2)

In [98]:
print(f"X_train's shape is {x_train.shape} , y_train's shape is {y_train.shape} , X_test's shape is {x_test.shape} , y_test's shape is {y_test.shape}")

X_train's shape is torch.Size([800, 2]) , y_train's shape is torch.Size([800]) , X_test's shape is torch.Size([200, 2]) , y_test's shape is torch.Size([200])


#2 Build or pick a pretrained model

To do so, we want to:
1. Setup device agonistic code so our code will run on an accelerator (GPU) if there is one


2. Construct a model (by subclassing nn.Module)


3. Define a loss function and optimizer


4. Create a training and test loop

In [99]:
#Make device agnostic code

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

Now we've setup device agnostic code, let's create a model that:
1. Subclasses nn. Module (almost all models in PyTorch subclass nn. Module)
2. Create 2 nn.Linear() layers that are capable of handling the shapes of our data
3. Defines a forward() method that outlines the forward pass (or forward computation) of the model
4. Instatiate an instance of our model class and send it to the target device

In [109]:
class CircleModel(nn.Module):
  #1.Construct a model that subclasses nn.Module
  def __init__(self):
    super().__init__()

    #2.Create 2 nn.Linear layers capable of handling the shapes of our data
    self.layer_1 = nn.Linear(in_features = 2 , out_features = 5)
    self.layer_2 = nn.Linear(in_features = 5 , out_features = 1)

    #3.Define a forward() method that outlines the forward pass
  def forward(self,x):
    return self.layer_2(self.layer_1(x)) #x -> layer_1 -> layer_2 -> output

#4.Instantiate instance of our model and send it to the target device
model = CircleModel()

In [110]:
model

CircleModel(
  (layer_1): Linear(in_features=2, out_features=5, bias=True)
  (layer_2): Linear(in_features=5, out_features=1, bias=True)
)

Random initialization of the model

In [111]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2484,  0.2194],
         [-0.1370,  0.0247],
         [ 0.3143, -0.6516],
         [ 0.2593,  0.3166],
         [ 0.1875, -0.0570]], requires_grad=True),
 Parameter containing:
 tensor([-0.2273, -0.0794, -0.4694, -0.1938, -0.0326], requires_grad=True),
 Parameter containing:
 tensor([[-0.1061,  0.0382, -0.3164,  0.1501,  0.3059]], requires_grad=True),
 Parameter containing:
 tensor([-0.1631], requires_grad=True)]

Replicating the above model using nn.Sequential()

In [114]:
model = nn.Sequential(
   nn.Linear(in_features = 2 , out_features = 5),
   nn.Linear(in_features = 5 , out_features = 1)
)

In [115]:
model.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.5416,  0.6249],
                      [ 0.2653, -0.5732],
                      [ 0.5224, -0.4944],
                      [-0.3869,  0.3043],
                      [ 0.0909, -0.4464]])),
             ('0.bias', tensor([-0.5720,  0.2178, -0.0676, -0.6792, -0.4941])),
             ('1.weight',
              tensor([[ 0.0943, -0.3979,  0.1752, -0.1429, -0.0223]])),
             ('1.bias', tensor([-0.2208]))])

Setting up Loss function and optimizer

In [118]:
loss_fn = nn.BCEWithLogitsLoss() #sigmoid activation function built-in

In [119]:
learning_rate = 0.001
optimizer = torch.optim.SGD(params = model.parameters() , lr = learning_rate)

#Train